In [1]:
p_ROI = "cc200"
p_fold = 10
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
a = torch.rand(8,4975)
a
a.shape

torch.Size([8, 4975])

In [3]:
!pip install pyprind

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
parameter_list = [p_ROI,p_fold]
print("*****List of patameters****")
print("ROI atlas: ", p_ROI)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pyprind
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn import tree
import functools
import numpy.ma as ma # for masked arrays
import pyprind
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

*****List of patameters****
ROI atlas:  cc200


## Importing the data 

In [ ]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

In [ ]:
data_main_path = "/content/drive/MyDrive/Abide_cc200/rois_"+ p_ROI #cc200'#path to time series data
flist = os.listdir(data_main_path)
print(len(flist))

for f in range(len(flist)):
    flist[f] = get_key(flist[f])
    

df_labels = pd.read_csv("/content/drive/MyDrive/preprocessing ABIDE iran CNN/data/phenotypes/Phenotypic_V1_0b_preprocessed1.csv")#path 
df_labels = df_labels[df_labels['FILE_ID'].str[0:4].isin(["OHSU"])] # only OHSU data
df_labels.DX_GROUP = df_labels.DX_GROUP.map({1: 1, 2:0})
print(len(df_labels))

labels = {}
for row in df_labels.iterrows():
    file_id = row[1]['FILE_ID']
    y_label = row[1]['DX_GROUP']
    if file_id == 'no_filename':
        continue
    assert(file_id not in labels)
    labels[file_id] = y_label

print(labels)


26
26
{'OHSU_0050142': 1, 'OHSU_0050143': 1, 'OHSU_0050144': 1, 'OHSU_0050145': 1, 'OHSU_0050146': 1, 'OHSU_0050147': 1, 'OHSU_0050148': 1, 'OHSU_0050149': 1, 'OHSU_0050150': 1, 'OHSU_0050152': 1, 'OHSU_0050153': 1, 'OHSU_0050156': 1, 'OHSU_0050157': 0, 'OHSU_0050158': 0, 'OHSU_0050159': 0, 'OHSU_0050160': 0, 'OHSU_0050161': 0, 'OHSU_0050162': 0, 'OHSU_0050163': 0, 'OHSU_0050164': 0, 'OHSU_0050166': 0, 'OHSU_0050167': 0, 'OHSU_0050168': 0, 'OHSU_0050169': 0, 'OHSU_0050170': 0, 'OHSU_0050171': 0}


### Helper functions for computing correlations

In [ ]:
def get_label(filename):
    assert (filename in labels)
    return labels[filename]


def get_corr_data(filename):
    #print(filename)
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
            
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()

def get_corr_matrix(filename):
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

def get_regs(samplesnames,regnum):
    datas = []
    for sn in samplesnames:
        datas.append(all_corr[sn][0])
    datas = np.array(datas)
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0)
    return regions


In [ ]:
if not os.path.exists('/content/drive/MyDrive/Abide_cc200/correlations_file'+p_ROI+'.pkl'):
    pbar=pyprind.ProgBar(len(flist))
    all_corr = {}
    for f in flist:
        lab = get_label(f)
        all_corr[f] = (get_corr_data(f), lab)
        pbar.update()

    print('Corr-computations finished')
    

    pickle.dump(all_corr, open('/content/drive/MyDrive/Abide_cc200/correlations_file'+p_ROI+'.pkl', 'wb'))
    print('Saving to file finished')

else:
    all_corr = pickle.load(open('/content/drive/MyDrive/Abide_cc200/correlations_file'+p_ROI+'.pkl', 'rb'))
    

In [ ]:
# CORRELATION MATRIX UPPER TRIANGLE is A with an array of shape (2,)
A = np.array(all_corr['OHSU_0050142'], dtype=object)
#A[0].shape
#all_corr['OHSU_0050142']
timeseries = np.array(A[0])
timeseries = timeseries.reshape(timeseries.shape[0],1)
timeseries.shape


(19900, 1)

## Getting the correlation matrix

In [ ]:
if not os.path.exists('/content/drive/MyDrive/Abide_cc200/correlations_file'+p_ROI+'.pkl'):
    pbar=pyprind.ProgBar(len(flist))
    all_corr = {}
    for f in flist:
        lab = get_label(f)
        all_corr[f] = (get_corr_data(f), lab)
        pbar.update()

    print('Corr-computations finished')
    

    pickle.dump(all_corr, open('/content/drive/MyDrive/Abide_cc200/correlations_file'+p_ROI+'.pkl', 'wb'))
    print('Saving to file finished')

else:
    all_corr = pickle.load(open('/content/drive/MyDrive/Abide_cc200/correlations_file'+p_ROI+'.pkl', 'rb'))


In [ ]:
total_correlations = pickle.load(open('/content/drive/MyDrive/temp data for ABIDE/correlations_file'+p_ROI+'.pkl', 'rb'))
len(total_correlations.get('Caltech_0051456')[0])
total_correlations.get('Caltech_0051456')[0].shape # (76636,) for cc400(4X)
#len(total_correlations)

(19900,)

## Computing eigenvalues and eigenvector

In [ ]:
eig_data = {}
pbar = pyprind.ProgBar(len(flist))
for f in flist:  
    d = get_corr_matrix(f)
    eig_vals, eig_vecs = np.linalg.eig(d)

    for ev in eig_vecs.T:
        np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))

    sum_eigvals = np.sum(np.abs(eig_vals))
    # Make a list of (eigenvalue, eigenvector, norm_eigval) tuples
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i], np.abs(eig_vals[i])/sum_eigvals)
                  for i in range(len(eig_vals))]

    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    eig_data[f] = {'eigvals':np.array([ep[0] for ep in eig_pairs]),
                    'norm-eigvals':np.array([ep[2] for ep in eig_pairs]),
                    'eigvecs':[ep[1] for ep in eig_pairs]}
    pbar.update()

0% [##########################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


## Defining dataset class

In [ ]:
class CC200Dataset(Dataset):
    def __init__(self, pkl_filename=None, data=None, samples_list=None, 
                 augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, verbose=False,regs=None):
        self.regs=regs
        if pkl_filename is not None:
            if verbose:
                print ('Loading ..!', end=' ')
            self.data = pickle.load(open(pkl_filename, 'rb'))
        elif data is not None:
            self.data = data.copy()
            
        else:
            sys.stderr.write('Eigther PKL file or data is needed!')
            return 

        #if verbose:
        #    print ('Preprocess..!', end='  ')
        if samples_list is None:
            self.flist = [f for f in self.data]
        else:
            self.flist = [f for f in samples_list]
        self.labels = np.array([self.data[f][1] for f in self.flist])
        
        current_flist = np.array(self.flist.copy())
        current_lab0_flist = current_flist[self.labels == 0]
        current_lab1_flist = current_flist[self.labels == 1]
        #if verbose:
        #    print(' Num Positive : ', len(current_lab1_flist), end=' ')
        #    print(' Num Negative : ', len(current_lab0_flist), end=' ')
        
        

        self.num_data = len(self.flist)

        # returns X and Y 
    def __getitem__(self, index):
        if index < len(self.flist):
            fname = self.flist[index]
            data = self.data[fname][0].copy() #get_corr_data(fname, mode=cal_mode)    
            data = data[self.regs].copy()
            label = (self.labels[index],)
            return torch.FloatTensor(data), torch.FloatTensor(label)
        else:
            f1 = self.flist[index % len(self.flist)]
            d1, y1 = self.data[f1][0], self.data[f1][1]
            d1=d1[self.regs]
            f2 = np.random.choice(self.neighbors[f1])
            d2, y2 = self.data[f2][0], self.data[f2][1]
            d2=d2[self.regs]
            assert y1 == y2
            r = np.random.uniform(low=0, high=1)
            label = (y1,)
            data = r*d1 + (1-r)*d2
            return torch.FloatTensor(data), torch.FloatTensor(label)

    def __len__(self):
        return self.num_data

## Definig data loader function

In [ ]:
def get_loader(pkl_filename=None, data=None, samples_list=None,
               batch_size=64, 
               num_workers=1, mode='train',
               *, augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, verbose=False,regions=None):
    """Build and return data loader."""
    if mode == 'train':
        shuffle = True
    else:
        shuffle = False
        augmentation=False

    dataset = CC200Dataset(pkl_filename=pkl_filename, data=data, samples_list=samples_list,
                           augmentation=augmentation, aug_factor=aug_factor, 
                           eig_data=eig_data, verbose=verbose,regs=regions)

    data_loader = DataLoader(dataset,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers)
    return data_loader

## Defining Autoencoder class

In [ ]:
class MTAutoEncoder(nn.Module):
    def __init__(self, num_inputs=990, 
                 num_latent=200, tied=True,
                 num_classes=2, use_dropout=False):
        super(MTAutoEncoder, self).__init__()
        self.tied = tied
        self.num_latent = num_latent
        
        self.fc_encoder = nn.Linear(num_inputs, num_latent) # encoder : 990 > 200
        #decoder if weights not tied
        if tied== False:  
            self.fc_decoder = nn.Linear(num_latent, num_inputs) 
         
        self.fc_encoder = nn.Linear(num_inputs, num_latent)
        #SLP with AE bottleneck as input 
        if use_dropout:
            self.classifier = nn.Sequential (
                nn.Dropout(p=0.4), # prob a neuron will be zeroed
                nn.Linear(self.num_latent, int(self.num_latent/2)),
                #nn.ReLU(),
                nn.Linear(int(self.num_latent/2), int(self.num_latent/10)),
                #nn.ReLU(),
                nn.Linear(int(self.num_latent/10), 1),                
            )
        else:
            self.classifier = nn.Sequential (
                nn.Linear(self.num_latent, int((self.num_latent)/2)),
                #nn.ReLU(),
                nn.Linear(int(self.num_latent/2), int(self.num_latent/10)),
                #nn.ReLU(),
                nn.Linear(int(self.num_latent/10), 1),
            )
            
         
    def forward(self, x, eval_classifier=False):
        x = self.fc_encoder(x)
        x = torch.tanh(x)
        if eval_classifier:
            x_logit = self.classifier(x)
        else:
            x_logit = None # initialised
        
        if self.tied: 
            x = F.linear(x, self.fc_encoder.weight.t())
        else:
            x = self.fc_decoder(x)
            
        return x, x_logit

mtae = MTAutoEncoder()

mtae

MTAutoEncoder(
  (fc_encoder): Linear(in_features=990, out_features=200, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=200, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=1, bias=True)
  )
)

## Defining training and testing functions


In [ ]:
def train(model, epoch, train_loader, p_bernoulli=None, mode='both', lam_factor=1.0):
    model.train()  
    train_losses = []
    for i,(batch_x,batch_y) in enumerate(train_loader):
        if len(batch_x) != batch_size:  # batch_size can't be equal to training size
            continue
        if p_bernoulli is not None:
            if i == 0:
                p_tensor = torch.ones_like(batch_x).to(device)*p_bernoulli
            rand_bernoulli = torch.bernoulli(p_tensor).to(device)

        data, target = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()

        if mode in ['both', 'ae']:
            if p_bernoulli is not None:
                rec_noisy, _ = model(data*rand_bernoulli, False)
                loss_ae = criterion_ae(rec_noisy, data) / len(batch_x)
            else:
                rec, _ = model(data, False)
                loss_ae = criterion_ae(rec, data) / len(batch_x)

        if mode in ['both', 'clf']:
            rec_clean, logits = model(data, True)
            loss_clf = criterion_clf(logits, target)

        if mode == 'both':
            loss_total = loss_ae + lam_factor*loss_clf
            train_losses.append([loss_ae.detach().cpu().numpy(), 
                                 loss_clf.detach().cpu().numpy()])
        elif mode == 'ae':
            loss_total = loss_ae
            train_losses.append([loss_ae.detach().cpu().numpy(), 
                                 0.0])
        elif mode == 'clf':
            loss_total = loss_clf
            train_losses.append([0.0, 
                                 loss_clf.detach().cpu().numpy()])

        loss_total.backward()
        optimizer.step()

    return train_losses       

def test(model, criterion, test_loader, 
         eval_classifier=False, num_batch=None):
    test_loss, n_test, correct = 0.0, 0, 0
    all_predss=[]
    if eval_classifier:
        y_true, y_pred = [], []
    with torch.no_grad():
        model.eval()
        for i,(batch_x,batch_y) in enumerate(test_loader, 1):
            if num_batch is not None:
                if i >= num_batch:
                    continue
            data = batch_x.to(device)
            rec, logits = model(data, eval_classifier)

            test_loss += criterion(rec, data).detach().cpu().numpy() 
            n_test += len(batch_x)
            if eval_classifier:
                proba = torch.sigmoid(logits).detach().cpu().numpy()
                preds = np.ones_like(proba, dtype=np.int32)
                preds[proba < 0.5] = 0
                all_predss.extend(preds)###????
                y_arr = np.array(batch_y, dtype=np.int32)

                correct += np.sum(preds == y_arr)
                y_true.extend(y_arr.tolist())
                y_pred.extend(proba.tolist())
        mlp_acc,mlp_sens,mlp_spef = confusion(y_true,all_predss)

    return  mlp_acc,mlp_sens,mlp_spef#,correct/n_test




In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
!nvidia-smi

Thu Mar 31 09:43:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    58W / 149W |    783MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:


num_corr = len(all_corr[flist[0]][0])
print("num_corr:  ",num_corr)

start =time.time()
batch_size = 8   #orginal = 8 
learning_rate_ae, learning_rate_clf = 0.0001, 0.001
num_epochs = 25   #original 25

p_bernoulli = None
use_dropout = True
augmentation = True

aug_factor = 2
num_neighbs = 5
lim4sim = 2
n_lat = int(num_corr/4)
print("this is n_lat",n_lat)
start= time.time()

print('p_bernoulli: ', p_bernoulli)
print('augmentaiton: ', augmentation, 'aug_factor: ', aug_factor, 
      'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
print('use_dropout: ', use_dropout, '\n')


crossval_res_kol=[]
all_rp_res = []
y_arr = np.array([get_label(f) for f in flist])
flist = np.array(flist)
kk=0 
for rp in range(10):
    kf = StratifiedKFold(n_splits=p_fold, random_state=1, shuffle=True)
    np.random.shuffle(flist)
    y_arr = np.array([get_label(f) for f in flist])
    for kk,(train_index, test_index) in enumerate(kf.split(flist, y_arr)):
        train_samples, test_samples = flist[train_index], flist[test_index]


        verbose = (True if (kk == 0) else False)

        regions_inds = get_regs(train_samples,int(num_corr/4))

        num_inpp = len(regions_inds)
        n_lat = int(num_inpp/2)
        train_loader=get_loader(data=all_corr, samples_list=train_samples, 
                                batch_size=batch_size, mode='train',
                                augmentation=augmentation, aug_factor=aug_factor, 
                                num_neighbs=num_neighbs, eig_data=eig_data, 
                                verbose=verbose,regions=regions_inds)
    #shape of train loader : image = [8,9950] and label = [8,1]

        test_loader=get_loader(data=all_corr, samples_list=test_samples, 
                                batch_size=batch_size, mode='test', augmentation=False, 
                                verbose=verbose,regions=regions_inds)
    #shape of test loader : image = [3,9950] and label = [3,1]

        model = MTAutoEncoder(tied=True, num_inputs=num_inpp, num_latent=n_lat, use_dropout=use_dropout)
        model.to(device)
        criterion_ae = nn.MSELoss(reduction='sum')
        criterion_clf = nn.BCEWithLogitsLoss()
        optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                                {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                              momentum=0.9)

        for epoch in range(1, num_epochs+1):
            if epoch <= 20:
                train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
            else:
                train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')


        res_mlp = test(model, criterion_ae, test_loader, eval_classifier=True)
        
        print(test(model, criterion_ae, test_loader, eval_classifier=True))
        crossval_res_kol.append(res_mlp)

    print("averages:")
    print(np.mean(np.array(crossval_res_kol),axis = 0))
    all_rp_res.append(np.mean(np.array(crossval_res_kol),axis = 0))
    finish= time.time()

    print(finish-start)
print("Avergae result of 10 repeats: ",np.mean(np.array(all_rp_res),axis = 0))



num_corr:   19900
this is n_lat 4975
p_bernoulli:  None
augmentaiton:  True aug_factor:  2 num_neighbs:  5 lim4sim:  2
use_dropout:  True 

(1.0, 1.0, 1.0)
(0.3333333333333333, 0.5, 0.0)
(0.6666666666666666, 1.0, 0.5)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(0.6666666666666666, 1.0, 0.5)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(0.5, 0.0, 1.0)
averages:
[0.81666667 0.85       0.8       ]
94.32975840568542
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(0.6666666666666666, 0.0, 1.0)
(1.0, 1.0, 1.0)
(0.6666666666666666, 1.0, 0.5)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(0.5, 0.0, 1.0)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
averages:
[0.85  0.825 0.875]
181.67562246322632
(0.6666666666666666, 0.5, 1.0)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(0.6666666666666666, 1.0, 0.5)
(0.6666666666666666, 1.0, 0.5)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(0.5, 1.0, 0.0)
averages:
[0.85       0.86666667 0.85      ]
269.14678168296814
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(1.0, 1.0, 1.0)
(0.6666666666666666,